In [46]:
import glob
import os
import pandas as pd
import ast


In [47]:
# Define the path to the folder containing the text files
folder_path = '../data/test_data'

# Use glob to get a list of all text files in the folder
text_files = glob.glob(os.path.join(folder_path, '*.txt'))

# Initialize lists to store data
vectors = []
clients = []
angles = []
parkinson_status = []

# Loop through the list and read each file
for file_path in text_files:
    # Extract client, angle, and Parkinson status from the file name
    file_name = os.path.basename(file_path).split('.')[0]
    parts = file_name.split('_')
    client = int(parts[0])
    angle = int(parts[1])
    parkinson = 1 if parts[2].lower() == 'parkinson' else 0
    
    with open(file_path, 'r') as file:
        for line in file:
            # Extract the vector from the line
            _, vector_str = line.split(': ')
            vector = ast.literal_eval(vector_str.strip())
            
            # Flatten the nested list if needed
            vector = [item for sublist1 in vector for sublist2 in sublist1 for item in sublist2]
            
            # Append to lists
            vectors.append(vector)
            clients.append(client)
            angles.append(angle)
            parkinson_status.append(parkinson)

# Create the DataFrame
data = {
    'client': clients,
    'angle': angles,
    'parkinson': parkinson_status,
    'vector': vectors
}
df = pd.DataFrame(data)

# Save DataFrame to a CSV file if needed
df.to_csv('output.csv', index=False)

,client,angle,parkinson,vector
0,1,0,1,"[[38.89351272583008, 17.675010681152344], [43...."
1,1,0,1,"[[85.9338607788086, 29.34123420715332], [92.69..."
2,1,0,1,"[[50.33542251586914, 20.45751190185547], [55.2..."
3,1,0,1,"[[44.83964920043945, 21.37966537475586], [50.3..."
4,1,0,1,"[[64.98465728759766, 29.381858825683594], [70...."


In [48]:
df

,client,angle,parkinson,vector
0,1,0,1,"[[38.89351272583008, 17.675010681152344], [43...."
1,1,0,1,"[[85.9338607788086, 29.34123420715332], [92.69..."
2,1,0,1,"[[50.33542251586914, 20.45751190185547], [55.2..."
3,1,0,1,"[[44.83964920043945, 21.37966537475586], [50.3..."
4,1,0,1,"[[64.98465728759766, 29.381858825683594], [70...."
...,...,...,...,...
546,1,90,1,"[[157.67861938476562, 44.677371978759766], [15..."
547,1,90,1,"[[104.82242584228516, 46.87606430053711], [109..."
548,1,90,1,"[[135.35787963867188, 49.76270294189453], [138..."
549,1,90,1,"[[133.68704223632812, 43.090797424316406], [13..."


In [50]:
#get the shortest vector length
min_len = min([len(v) for v in df['vector']])
min_len

17

In [49]:
df.to_csv('../output_data/output_2D.csv', index=False)

In [27]:
import numpy as np
import torch

In [41]:
# Function to add a third element 0 to each sublist
def add_third_element(vector):
    return [sublist + [0] for sublist in vector[0][0]]

In [42]:
# Function to process each vector
def process_vector(vector):
    # Add third element 0 to each sublist
    vector = add_third_element(vector)
    
    # Convert to numpy array
    data = np.array(vector)
    
    # Add batch and frame dimensions
    data = data[np.newaxis, np.newaxis, :, :]
    
    # Add Z coordinate (set to 0 for 2D data)
    data = np.pad(data, ((0,0), (0,0), (0,0), (0,0), (0,1)), mode='constant')
    
    # Reshape to [batch_size, num_channels, num_frames, num_joints]
    data = np.transpose(data, (0, 4, 1, 3, 2))
    
    # Repeat the frame 300 times to simulate a sequence
    data = np.repeat(data, 300, axis=2)
    
    # Convert to PyTorch tensor
    data_tensor = torch.FloatTensor(data)
    
    return data_tensor

[38.89351272583008, 17.675010681152344]

In [44]:
# Apply the function to the 'vector' column
df['vector'] = df['vector'].apply(add_third_element)

# Print the DataFrame to check the result
print(df)

     client  angle  parkinson  \
0         1      0          1   
1         1      0          1   
2         1      0          1   
3         1      0          1   
4         1      0          1   
..      ...    ...        ...   
546       1     90          1   
547       1     90          1   
548       1     90          1   
549       1     90          1   
550       1     90          1   

                                                vector  
0    [[38.89351272583008, 17.675010681152344, 0, 0]...  
1    [[85.9338607788086, 29.34123420715332, 0, 0], ...  
2    [[50.33542251586914, 20.45751190185547, 0, 0],...  
3    [[44.83964920043945, 21.37966537475586, 0, 0],...  
4    [[64.98465728759766, 29.381858825683594, 0, 0]...  
..                                                 ...  
546  [[157.67861938476562, 44.677371978759766, 0, 0...  
547  [[104.82242584228516, 46.87606430053711, 0, 0]...  
548  [[135.35787963867188, 49.76270294189453, 0, 0]...  
549  [[133.68704223632812, 43.090

In [45]:
df.to_csv('../output_data/output.csv', index=False)